# Imports

In [1]:
import csv
import numpy as np
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn import metrics
import math

# Suppress Warnings
import warnings
warnings.filterwarnings('ignore')

# Preprocessing

In [2]:
#========== IMPORTS =============#
# Allows jupyter notebook to be imported
import jupyter_import
from data_preproc.Preprocess import preprocess, Normalize
#================================#

importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\data_preproc\Preprocess.ipynb


# Feature Selection

In [3]:
from data_preproc.CFS import cfs_algo
from data_preproc.RFE import rfe_algo
from data_preproc.RR import ridge_algo

importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\data_preproc\CFS.ipynb
importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\data_preproc\RFE.ipynb
importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\data_preproc\RR.ipynb


# Algorithms

## Base Predictors

In [4]:
from pred_mdls.base.Complement_Naive_Bayes import complement_naive_bayes_model
from pred_mdls.base.Decision_Tree import decision_tree_model
from pred_mdls.base.Logistic_Regression import logistic_regression_model
from pred_mdls.base.Multi_Layer_Perceptron import multi_layer_perceptron_model
from pred_mdls.base.Naive_Bayes import naive_bayes_model

importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\pred_mdls\base\Complement_Naive_Bayes.ipynb
importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\pred_mdls\base\Decision_Tree.ipynb
importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\pred_mdls\base\Logistic_Regression.ipynb
importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\pred_mdls\base\Multi_Layer_Perceptron.ipynb
importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\pred_mdls\base\Naive_Bayes.ipynb


## Ensemble Predictors

In [5]:
from pred_mdls.ensemble.Random_Forest import random_forest_model
from pred_mdls.ensemble.Rotation_Forest import rotation_forest_model
from pred_mdls.ensemble.Voting import voting_model

importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\pred_mdls\ensemble\Random_Forest.ipynb
importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\pred_mdls\ensemble\Rotation_Forest.ipynb
importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\pred_mdls\ensemble\Voting.ipynb


# Evaluation Metrics

In [6]:
from pf_eval.AUC_ROC import auc_roc_model
from pf_eval.F1_Score import f1_model
from pf_eval.CSV import write_results

importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\pf_eval\AUC_ROC.ipynb
importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\pf_eval\F1_Score.ipynb
importing Jupyter notebook from c:\Monash University\FIT3162 Computer Science Project 2\Program\Algorithm\pf_eval\CSV.ipynb


In [8]:
def data_conversion(data):
    for i in range(len(data)):
        if data[i] == b'N' or data[i] == b'false':
            data[i] = 0
        else:
            data[i] = 1
    return data

def read_data(filename):
    data = arff.loadarff(filename)
    loaddata = pd.DataFrame(data[0])
    return loaddata

def process_data(loaddata,features):
    # Features are selected based on CFS
    software_metrics = np.array(loaddata[features])
    labels = np.array(loaddata['Defective'])
    return software_metrics,labels

def train_data(software_metrics,labels):
    X_train, X_test, y_train, y_test = train_test_split(software_metrics, labels, test_size = 0.1)
    y_train = y_train.astype('str')
    y_test = y_test.astype('str')
    return X_train, X_test, y_train, y_test

def evaluate_data(model,model_name,X_test,y_test):
    predictions = model.predict(X_test)
    auc_score = round(auc_roc_model(model,X_test,y_test),2)
    f1_score = round(f1_model(model,X_test,y_test),2)
    #print(f"Model Name: {model_name}")
    #print(f'Accuracy: {round(metrics.accuracy_score(y_test, predictions)*100,2)}%')
    #print(f'AUC Score: {auc_score}')
    #print(f'F1-score: {f1_score}')
    return auc_score,f1_score

def translate(result):
    count = 1
    res = []
    while count <= 3:
        for i in range(len(result[0])):
            res.append([result[0][i], result[1][((i+1)*count)-1],result[2][((i+1)*count)-1]])
        count += 1
    return res

def write_results_S(header,result):
    filters = ['No filter','CFS','RFE']
    #header = ['Dataset', 'Algo 1', 'Algo 2','Algo 3']
    #dummy = [['CM1','95%','92%','87%'], ['KC1','Trying','To','Test'],['KC4','90%','80%','70%']]
    with open('pred_results.csv','w',encoding='UTF8', newline='') as file:
        res = csv.writer(file)
        for i in range(len(filters)):
            res.writerow('')
            res.writerow([filters[i]])
            res.writerow(header)
            res.writerow([result[0][0]] + result[0][1][i*8:i*8+8])
            res.writerow([result[1][0]] + result[1][1][i*8:i*8+8])
    
def run(datasets, savename):
    #result, header = main(filename)
    #output = write_results_M(header,result)
    header = ['Model name','Complement Naive Bayes','Decision Tree','Logistic regression',
                        'Multi Layer Perceptron','Naive Bayes','Random Forest','Rotation Forest','Voting']
    filters = ['(All)','(CFS)','(RFE)']
    #header = ['Dataset', 'Algo 1', 'Algo 2','Algo 3']
    #dummy = [['CM1','95%','92%','87%'], ['KC1','Trying','To','Test'],['KC4','90%','80%','70%']]
    with open(savename,'w',encoding='UTF8', newline='') as file:
        results = []
        for ds in datasets:
            results.append(main('datasets/NASA/' + ds + '.txt')[0])
        res = csv.writer(file)
        res.writerow([results[0][0][0]])
        res.writerow(header)
        for j in range(len(results)):
            for i in range(len(filters)):
                res.writerow([datasets[j]+filters[i]] + results[j][0][1][i*8:i*8+8])
        res.writerow('')
        res.writerow([results[0][1][0]])
        res.writerow(header)
        for j in range(len(results)):
            for i in range(len(filters)):
                res.writerow([datasets[j]+filters[i]] + results[j][1][1][i*8:i*8+8])
    
def main(filename):
    print(filename)
    # Read the file
    loaddata = read_data(filename)
    loaddata = Normalize(loaddata)
    #software_metrics = loaddata.iloc[:,:-1] #Software metrics
    #labels = loaddata.iloc[:,-1] #Labels
    SM = np.array(loaddata.iloc[:,:-1]) #Software metrics
    L = data_conversion(np.array(loaddata.iloc[:,-1])).astype(int) #Labels
    data = [SM,L]
    #print(data)
    # ===== Feature Selection ====== #

    # ==== CFS ==== #
    cfs, cfs_selections = cfs_algo(data,10)
    # ============= #

    # ===== RFE ======== #
    rfe, rfe_selections = rfe_algo(data,10)
    # ================== #
    
    # ========= Preprocessing ============= #
    pp = preprocess(loaddata)
    pp_cfs = preprocess(loaddata, cfs_selections)
    pp_rfe = preprocess(loaddata, rfe_selections)

    pp_arr = [pp,pp_cfs,pp_rfe]
    pp_name = ['No filters','CFS Feature Selection','RFE Feature Selection']
    args = [1000]
    # ====================================== #

    result = []
    auc_arr = [0]*24
    f1_arr = [0]*24
    header = []
    folds = 5
    for j,pp in enumerate(pp_arr):
        #print(pp_name[i])
        for i in range(folds):
            data = [pp[i][0],pp[i][2]]

            # ======== Model Creation =========== #
            # Base Predictors
            cnb = complement_naive_bayes_model(data,args)
            dt = decision_tree_model(data,args)
            lr = logistic_regression_model(data,args)
            mlp = multi_layer_perceptron_model(data,args)
            nb = naive_bayes_model(data,args)

            # Ensemble Predictors
            rf = random_forest_model(data,args)
            rof = rotation_forest_model(data,args)
            vt = voting_model(data,args)
            # ==================================== #

            models = [cnb,dt,lr,mlp,nb,rf,rof,vt] 
            model_name = ['Complement Naive Bayes','Decision Tree','Logistic regression',
                            'Multi Layer Perceptron','Naive Bayes','Random Forest','Rotation Forest','Voting']
            for k in range(len(models)):
                #print('*'*50)
                auc_score,f1_score = evaluate_data(models[k],model_name[k],pp[i][1],pp[i][3])
                if math.isnan(auc_score):
                    print(model_name[k], auc_score)
                    auc_score = 0
                # if math.isnan(f1_score):
                #     f1_score = 0
                auc_arr[(j*8)+k] += auc_score
                f1_arr[(j*8)+k] += f1_score
            #print('='*100)
    for i in range(len(models)*3):
        auc_arr[i] /= folds
        f1_arr[i] /= folds
    header.append('Model Name')
    header += model_name
    #result.append(model_name)
    result.append(('AUC', auc_arr))
    result.append(('F1 Score', f1_arr))
    return result, header     
            
if __name__=='__main__':
    #filename = 'datasets/NASA/KC4.arff.txt'
    #'CM1.arff','JM1.arff','KC1.arff','KC4.arff','MC2.arff','PC1.arff'
    N_filenames = ['MW1.arff','CM1.arff','JM1.arff','KC1.arff','KC4.arff','MC2.arff','PC1.arff','PC3.arff',
               'PC4.arff','PC5.arff']
    P_filenames = ['cm1.arff','jm1.arff','kc1.arff','kc2.arff','pc1.arff']


    # AUC
    # Take dataset not in the list (MW1.arff)

    # F1
    # CM1, JM1

    #for i in range(len(N_filenames)):
    #    N_filenames[i] = 'datasets/NASA/' + N_filenames[i] + '.txt'
    #for i in range(len(P_filenames)):
    #    P_filenames[i] = 'datasets/PROMISE/' + P_filenames[i] + '.txt'
    #result, header = main(filename)
    #output = write_results_S(header,result)
    run(N_filenames, 'NASA.csv')
    #print(header)
    #print(result)

datasets/NASA/MW1.arff.txt


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.